…a pak Hynek Cígler napíše, že čím méně filmů do srovnání bude, tím bude průkaznější. Samozřejmě. Takže se to profiltrujem podle počtu hodnocení na ČSFD a trochu i růčo.

In [351]:
import os
import polars as pl
import polars.selectors as cs
import altair as alt
import warnings

pl.Config(tbl_rows=100)

df = pl.read_parquet(os.path.join("data","fdb.parquet"))
vanocni = pl.read_json(os.path.join("data","vanocni_filmy_kompletni_metadata.json"))

from src.kristi_promin import kristi_promin
from src.me_to_neurazi import me_to_neurazi
from src.alt_friendly import alt_friendly

pl.Config.set_tbl_rows(150)
pl.Config.set_fmt_str_lengths(150)
pl.Config.set_tbl_width_chars(200)

alt.themes.register('irozhlas', kristi_promin)
alt.themes.enable('irozhlas')
warnings.filterwarnings("ignore")

In [352]:
with open("data_raw/vanocni_filmy_csfd.json", "r", encoding="utf-8") as imp:
    vanocni = json.loads(imp.read())
vanocni = pl.DataFrame(vanocni).transpose().rename({"column_0":"nazev",'column_1':'url','column_2':'format'})
vanocni

nazev,url,format
str,str,str
"""Adéla ještě nevečeřela""","""https://www.csfd.cz/film/5981-adela-jeste-nevecerela/prehled/""","""film"""
"""Anděl Páně""","""https://www.csfd.cz/film/215414-andel-pane/prehled/""","""film"""
"""Ať přiletí čáp, královno!""","""https://www.csfd.cz/film/108859-at-prileti-cap-kralovno/prehled/""","""tv"""
"""Bohouš""","""https://www.csfd.cz/film/8186-bohous/prehled/""","""tv"""
"""Byl jednou jeden král...""","""https://www.csfd.cz/film/10089-byl-jednou-jeden-kral/prehled/""","""film"""
"""Cesta do Ameriky""","""https://www.csfd.cz/film/5577-cesta-do-ameriky/prehled/""","""film"""
"""Co takhle svatba, princi?""","""https://www.csfd.cz/film/108854-co-takhle-svatba-princi/prehled/""","""tv"""
"""Císař a tambor""","""https://www.csfd.cz/film/5345-cisar-a-tambor/prehled/""","""film"""
"""Císařův pekař""","""https://www.csfd.cz/film/3094-cisaruv-pekar-pekaruv-cisar/prehled/""","""film"""


In [353]:
with open("data_raw/vanocni_filmy_od_petra.json", "r", encoding="utf-8") as imp:
    vanocni_od_petra = json.loads(imp.read())
vanocni_od_petra = pl.DataFrame(vanocni_od_petra).select(pl.col(['id','name'])).rename({'name':'nazev'})
vanocni_od_petra

id,nazev
i64,str
0,"""Adéla ještě nevečeřela"""
1,"""Anděl Páně"""
2,"""Bohouš"""
3,"""Byl jednou jeden král..."""
4,"""Dalskabáty, hříšná ves aneb Zapomenutý čert"""
5,"""Dařbuján a Pandrhola"""
6,"""Dobrý voják Švejk"""
7,"""Grinch"""
8,"""Láska nebeská"""


In [354]:
vanocni_od_petra.select(pl.col("nazev")).to_series().to_list()

['Adéla ještě nevečeřela',
 'Anděl Páně',
 'Bohouš',
 'Byl jednou jeden král...',
 'Dalskabáty, hříšná ves aneb Zapomenutý čert',
 'Dařbuján a Pandrhola',
 'Dobrý voják Švejk',
 'Grinch',
 'Láska nebeská',
 'Láska rohatá',
 'Mrazík',
 'Na samotě u lesa',
 'Nesmrtelná teta',
 'Někdo to rád horké',
 'Pelíšky',
 'Pretty Woman',
 'Princ a Večernice',
 'Princezna se zlatou hvězdou',
 'Prázdniny v Římě',
 'Pyšná princezna',
 'S tebou mě baví svět',
 'S čerty nejsou žerty',
 'Shrek',
 'Skála',
 'Sám doma',
 'Císařův pekař',
 'Tři oříšky pro Popelku',
 'Veselé Vánoce přejí chobotnice',
 'Z pekla štěstí',
 'Šíleně smutná princezna']

In [355]:
animace = [
    'Adéla ještě nevečeřela',
    'Grinch',
    'Shrek',
    'Veselé Vánoce přejí chobotnice',
]

travesti = [
    "Někdo to rád horké",
    'Tři oříšky pro Popelku',
    'S tebou mě baví svět',
    'Mrazík'
]

nadprirozene = [
    'Adéla ještě nevečeřela',
 'Anděl Páně',
    'Byl jednou jeden král...',
 'Dalskabáty, hříšná ves aneb Zapomenutý čert',
 'Dařbuján a Pandrhola',
    'Grinch',
    'Láska rohatá',
 'Mrazík',
    'Nesmrtelná teta',
    'Princ a Večernice',
 'Princezna se zlatou hvězdou',
    'S čerty nejsou žerty',
 'Shrek',
    'Císařův pekař',
 'Tři oříšky pro Popelku',
 'Veselé Vánoce přejí chobotnice',
 'Z pekla štěstí',
    'Pyšná princezna'
]

jesex = [
    'Někdo to rád horké',
    'Pretty Woman',
    'Pelíšky',
    'Láska nebeská',
    'S tebou mě baví svět',
    'Skála'
]

In [356]:
vanocni_detaily = pl.scan_ndjson("data_raw/csfd/*.json").collect().rename({'titul':'nazev'})

vanocni_detaily.sort(by="pocet_hodnoceni",descending=True)

id,nazev,zanry,zeme,rok,delka,rezie,scenar,hraji,anotace,hodnoceni,pocet_hodnoceni
str,str,list[str],str,i64,i64,list[str],list[str],list[str],str,i64,i64
"""4570-pelisky""","""Pelíšky""","[""komedie"", ""drama""]","""Česko""",1999,116,"[""Jan Hřebejk""]","[""Petr Jarchovský""]","[""Miroslav Donutil"", ""Jiří Kodet"", … ""více""]","""Příběhy jedné historické generace - stárnoucích rodičů, dospívající mládeže a malých dětí. Děj je zasazen do konce šedesátých let - podzim 67 až léto …",91,117084
"""14999-shrek""","""Shrek""","[""animovaný"", ""dobrodružný"", … ""komedie""]","""USA""",2001,90,"[""Vicky Jenson"", ""Andrew Adamson""]","[""Ted Elliott"", ""Terry Rossio"", … ""Roger S.H. Schulman""]","[""Mike Myers"", ""Eddie Murphy"", … ""více""]","""Shrek, zelený obr s tykadly mimozemšťanů, žije sice docela sám, ale šťastně ve svém domku na samotě u lesa až do chvíle, kdy je doslova přepaden nekon…",88,101749
"""821-s-certy-nejsou-zerty""","""S čerty nejsou žerty""","[""pohádka"", ""komedie""]","""Československo""",1984,91,"[""Hynek Bočan""]","[""Jiří Just"", ""Hynek Bočan""]","[""Luděk Munzar"", ""Vladimír Dlouhý"", … ""více""]","""Chamtivá macecha připraví svého nevlastního syna Petra o všechno, včetně dědictví po jeho otci. Petr se mezitím seznámí s dcerami místního knížete, kd…",89,80866
"""1628-sam-doma""","""Sám doma""","[""rodinný"", ""komedie""]","""USA""",1990,103,"[""Chris Columbus""]","[""John Hughes""]","[""Macaulay Culkin"", ""Joe Pesci"", … ""více""]","""""Mám pocit, že jsme doma nechali něco důležitého,"" sdělí paní McCallisterová manželovi na palubě letadla směřujícího přes Atlantik do Paříže. To ještě…",86,80771
"""43677-laska-nebeska""","""Láska nebeská""","[""komedie"", ""drama"", ""romantický""]","""Velká Británie / USA / Francie""",2003,135,"[""Richard Curtis""]","[""Richard Curtis""]","[""Bill Nighy"", ""Gregor Fisher"", … ""více""]","""Láska nebeská, romantická komedie ve stylu Čtyři svatby a jeden pohřeb, Notting Hill a Deník Bridget Jonesové, nás zavede do současného předvánočního …",85,76558
"""9430-tri-orisky-pro-popelku""","""Tři oříšky pro Popelku""","[""pohádka""]","""Československo / Východní Německo""",1973,75,"[""Václav Vorlíček""]","[""František Pavlíček"", ""Václav Vorlíček""]","[""Libuše Šafránková"", ""Pavel Trávníček"", … ""více""]","""Popelka je milé hodné děvče bez matky a posléze i bez otce, který ji nechá napospas své druhé ženě a její dceři. Obě nemají Popelku rády a mimo jiné s…",86,73324
"""15000-shrek-2""","""Shrek 2""","[""animovaný"", ""dobrodružný"", … ""komedie""]","""USA""",2004,93,"[""Conrad Vernon"", ""Andrew Adamson"", ""Kelly Asbury""]","[""Andrew Adamson"", ""Joe Stillman"", … ""David N. Weiss""]","[""Mike Myers"", ""Eddie Murphy"", … ""více""]","""A byla svatba a pak spolu Shrek s Fionou žili šťastně až do smrti… Tedy žili by, kdyby se ovšem nevydali za princezninými rodiči! Návštěva království …",83,66882
"""6663-na-samote-u-lesa""","""Na samotě u lesa""","[""komedie"", ""rodinný"", ""drama""]","""Československo""",1976,93,"[""Jiří Menzel""]","[""Zdeněk Svěrák"", ""Ladislav Smoljak""]","[""Josef Kemr"", ""Zdeněk Svěrák"", … ""více""]","""Hrdiny jsou manželé Oldřich a Věra Lavičkovi, kteří se svými dětmi Petříkem a Zuzankou odjíždějí do Loukova na chalupu svého známého, inženýra Radima …",88,66837
"""10089-byl-jednou-jeden-kral""","""Byl jednou jeden král...""","[""pohádka"", ""komedie""]","""Československo""",1954,107,"[""Bořivoj Zeman""]","[""Jan Werich"", ""Jiří Brdečka"", ""Bořivoj Zeman""]","[""Jan Werich"", ""Vlasta Burian"", … ""více""]","""Byl jednou jeden král a ten si říkal Já První. Chtěl odevzdat vládu té ze tří dcer, která ho má nejraději. Drahomíra ho má ráda jako zlato, Zpěvanka j…",87,56191


In [357]:
df.sample(1)

kanal,datum,nazev,extra,noticka,url
cat,datetime[μs],str,str,str,str
"""ČT2""",2011-05-31 05:00:00,"""Příště u Vás z Čáslavi""","""Hudebně zábavný pořad""",null,null


In [358]:
vanocni.sample(1)

nazev,url,format
str,str,str
"""To neznáte Hadimršku""","""https://www.csfd.cz/film/3166-to-neznate-hadimrsku/prehled/""","""film"""


In [359]:
vanocni_detaily

id,nazev,zanry,zeme,rok,delka,rezie,scenar,hraji,anotace,hodnoceni,pocet_hodnoceni
str,str,list[str],str,i64,i64,list[str],list[str],list[str],str,i64,i64
"""10086-nesmrtelna-teta""","""Nesmrtelná teta""","[""pohádka""]","""Česko""",1993,98,"[""Zdeněk Zelenka""]","[""Zdeněk Zelenka""]","[""Jiřina Bohdalová"", ""Jaromír Hanzlík"", … ""více""]","""Temperamentní situační pohádková komedie o vesnickém chasníkovi Matějovi, který náhle pobral rozum a rozhodl se usilovat o ruku princezny Pavlínky, je…",63,23249
"""10089-byl-jednou-jeden-kral""","""Byl jednou jeden král...""","[""pohádka"", ""komedie""]","""Československo""",1954,107,"[""Bořivoj Zeman""]","[""Jan Werich"", ""Jiří Brdečka"", ""Bořivoj Zeman""]","[""Jan Werich"", ""Vlasta Burian"", … ""více""]","""Byl jednou jeden král a ten si říkal Já První. Chtěl odevzdat vládu té ze tří dcer, která ho má nejraději. Drahomíra ho má ráda jako zlato, Zpěvanka j…",87,56191
"""10094-pysna-princezna""","""Pyšná princezna""","[""pohádka"", ""komedie"", … ""romantický""]","""Československo""",1952,94,"[""Bořivoj Zeman""]","[""Bořivoj Zeman"", ""Henryk Bloch"", ""Oldřich Kautský""]","[""Alena Vránová"", ""Vladimír Ráž"", … ""více""]","""Pohádka všech českých pohádek o pyšné princezně Krasomile, která odmítla vzít si za muže krále Miroslava. Ten si to však nenechal líbit a v přestrojen…",82,28205
"""10096-silene-smutna-princezna""","""Šíleně smutná princezna""","[""pohádka"", ""hudební"", ""komedie""]","""Československo""",1968,89,"[""Bořivoj Zeman""]","[""František Vlček st."", ""Bořivoj Zeman""]","[""Helena Vondráčková"", ""Václav Neckář"", … ""více""]","""Hudební pohádka Šíleně smutná princezna je určena především malým - ovšem ne těm nejmenším divákům. Vypráví příběh prince a princezny ze sousedních sp…",83,32665
"""108846-o-medvedu-ondrejovi""","""O medvědu Ondřejovi""","[""pohádka"", ""komedie""]","""Československo""",1959,50,"[""Jaroslav Mach""]",null,"[""Jaroslav Marvan"", ""Aglaia Morávková"", … ""více""]","""Princezna Blanka se tajně schází se svým milým, myslivcem Ondřejem. Jejich lásce přeje jen komorná Anežka, královský otec se nesmí nic dozvědět. Blank…",59,1851
"""108854-co-takhle-svatba-princi""","""Co takhle svatba, princi?""","[""pohádka"", ""hudební""]","""Československo""",1986,90,"[""Jiří Adamec""]","[""Jiří Chalupa""]","[""Miroslav Vladyka"", ""Ivana Andrlová"", … ""více""]","""Kdo by neznal jednu z nejznámějších filmových písniček ""Když se načančám"". Tuto a další písničky (např. ""Taková jsem já"", ""Princezny jdou si hajnout"" …",69,7403
"""108859-at-prileti-cap-kralovno""","""Ať přiletí čáp, královno!""","[""pohádka"", ""hudební""]","""Československo""",1988,93,"[""Jiří Adamec""]","[""Jiří Chalupa""]","[""Miroslav Vladyka"", ""Ivana Andrlová"", … ""více""]","""Ve volném pokračování pohádkové komedie Co takhle svatba, princi? se znovu setkáte s princem Davidem a jeho ženou Bělou. Oba se těší na děťátko, které…",61,5954
"""111222-kotva-u-privozu""","""Kotva u přívozu""","[""komedie"", ""drama""]","""Československo""",1980,90,"[""Marie Poledňáková""]","[""Marie Poledňáková""]","[""Jana Šulcová"", ""Petr Haničinec"", … ""více""]","""Magda je televizní hlasatelka. Pro miliony diváků příjemná, sympatická tvář na obrazovce. Klidná, úsměvná, vyrovnaná. Příchod pohody do rodinného veče…",60,2020
"""121859-princove-jsou-na-draka""","""Princové jsou na draka""","[""pohádka"", ""hudební""]","""Československo""",1980,57,"[""Jiří Adamec""]","[""Jiří Chalupa""]","[""Vladimír Menšík"", ""Ivana Andrlová"", … ""více""]","""Vtipný příběh o lásce a moudrosti, z kterého vyplývá, že nad drakem mohou zvítězit nejen princové. (oficiální text distributora)""",74,11814


In [360]:
vanocni_final = vanocni.with_columns(
    pl.col('nazev').replace({'Císařův pekař - Pekařův císař':'Císařův pekař'})
).join(
    vanocni_detaily.with_columns(
        pl.col("nazev").replace({'Císařův pekař - Pekařův císař':'Císařův pekař'})
    ),
    how='right',
    on='nazev'
).sort(
    by='hodnoceni', descending=True
).filter(
    ~pl.col("url").is_null()
).rename(
    {'id':'id_puvodni'}
).join(
    vanocni_od_petra,
    on='nazev',
    how='left'
)

In [361]:
vanocni_final.filter(pl.col("nazev").str.contains("Císařův"))

url,format,id_puvodni,nazev,zanry,zeme,rok,delka,rezie,scenar,hraji,anotace,hodnoceni,pocet_hodnoceni,id
str,str,str,str,list[str],str,i64,i64,list[str],list[str],list[str],str,i64,i64,i64
"""https://www.csfd.cz/film/3094-cisaruv-pekar-pekaruv-cisar/prehled/""","""film""","""3094-cisaruv-pekar-pekaruv-cisar""","""Císařův pekař""","[""komedie"", ""historický""]","""Československo""",1951,80,"[""Martin Frič""]","[""Jan Werich"", ""Martin Frič"", … ""více""]","[""Jan Werich"", ""Marie Vášová"", … ""více""]","""Historická veselohra o císaři Rudolfovi II., pekaři Matějovi a legendárním Golemovi. Tento příběh neodpovídá přesně historické pravdě, ale je zato nad…",86,55421,25


In [362]:
vanocni_final.filter(pl.col('id').is_null())

url,format,id_puvodni,nazev,zanry,zeme,rok,delka,rezie,scenar,hraji,anotace,hodnoceni,pocet_hodnoceni,id
str,str,str,str,list[str],str,i64,i64,list[str],list[str],list[str],str,i64,i64,i64
"""https://www.csfd.cz/film/5475-duchacek-to-zaridi/prehled/""","""film""","""5475-duchacek-to-zaridi""","""Ducháček to zařídí""","[""komedie""]","""Československo""",1938,86,"[""Karel Lamač""]","[""Václav Wasserman"", ""Rudolf Österreicher"", … ""více""]","[""Vlasta Burian"", ""Ladislav Hemmer"", … ""více""]","""Pan Ducháček je sice poněkud svérázný patron, nicméně jako koncipient advokátní kanceláře doktora Faulknera je naprosto nenahraditelný. Zejména v okam…",86,21640,null
"""https://www.csfd.cz/film/15000-shrek-2/prehled/""","""film""","""15000-shrek-2""","""Shrek 2""","[""animovaný"", ""dobrodružný"", … ""komedie""]","""USA""",2004,93,"[""Conrad Vernon"", ""Andrew Adamson"", ""Kelly Asbury""]","[""Andrew Adamson"", ""Joe Stillman"", … ""David N. Weiss""]","[""Mike Myers"", ""Eddie Murphy"", … ""více""]","""A byla svatba a pak spolu Shrek s Fionou žili šťastně až do smrti… Tedy žili by, kdyby se ovšem nevydali za princezninými rodiči! Návštěva království …",83,66882,null
"""https://www.csfd.cz/film/3083-anton-spelec-ostrostrelec/prehled/""","""film""","""3083-anton-spelec-ostrostrelec""","""Anton Špelec, ostrostřelec""","[""komedie""]","""Československo""",1932,80,"[""Martin Frič""]","[""Josef Neuberg"", ""Emil Artur Longen""]","[""Vlasta Burian"", ""Růžena Šlemrová"", … ""více""]","""Nejzdařilejší ze série burianovských komedií, natočených v třicátých letech, pojednává o tom, co způsobila urážka císařského majestátu, a o snaze vyhn…",82,10407,null
"""https://www.csfd.cz/film/3115-katakomby/prehled/""","""film""","""3115-katakomby""","""Katakomby""","[""komedie""]","""Protektorát Čechy a Morava""",1940,80,"[""Martin Frič""]","[""Václav Wasserman"", ""Gustav Davis""]","[""Vlasta Burian"", ""Jaroslav Marvan"", … ""více""]","""Starý úředník, oficiál Borman, úřaduje ve sklepních kancelářích Pozemkového úřadu společně s dalšími oficiály a aktuáry. Mezi jeho kolegy je i mladý a…",79,5661,null
"""https://www.csfd.cz/film/3166-to-neznate-hadimrsku/prehled/""","""film""","""3166-to-neznate-hadimrsku""","""To neznáte Hadimršku""","[""komedie""]","""Československo""",1931,91,"[""Karel Lamač"", ""Martin Frič""]","[""Václav Wasserman"", ""Franz Arnold"", … ""více""]","[""Vlasta Burian"", ""Meda Valentová"", … ""více""]","""Revident Hadimrška je vyslán do Prahy. Má reorganizovat Zlatníkovu gramofonovou firmu, aby mohla bance zaplatit dluhy. Mladý majitel se zruinoval vydr…",79,9278,null
"""https://www.csfd.cz/film/4238-panna-a-netvor/prehled/""","""film""","""4238-panna-a-netvor""","""Panna a netvor""","[""pohádka"", ""horor"", … ""romantický""]","""Československo""",1978,88,"[""Juraj Herz""]","[""Juraj Herz"", ""Ota Hofman"", ""František Hrubín""]","[""Zdena Studenková"", ""Vlastimil Harapes"", … ""více""]","""Slavnou strašidelnou pohádku, původně francouzskou, o princi začarovaném do děsivé zvířecí podoby, jehož vysvobodí až čistá láska, využil režisér Jura…",78,10311,null
"""https://www.csfd.cz/film/9411-jak-se-budi-princezny/prehled/""","""film""","""9411-jak-se-budi-princezny""","""Jak se budí princezny""","[""pohádka""]","""Československo""",1977,83,"[""Václav Vorlíček""]","[""Bohumila Zelenková""]","[""Jiří Sovák"", ""Milena Dvorská"", … ""více""]","""Ještě před narozením královské dcery jedou král s královnou do zpustlé tvrze - v ní žije královnina sestra, už spoustu let odloučena od světa. Když si…",78,24890,null
"""https://www.csfd.cz/film/5577-cesta-do-ameriky/prehled/""","""film""","""5577-cesta-do-ameriky""","""Cesta do Ameriky""","[""komedie"", ""romantický""]","""USA""",1988,116,"[""John Landis""]","[""David Sheffield"", ""Barry W. Blaustein""]","[""Eddie Murphy"", ""Arsenio Hall"", … ""více""]","""Král komedie Eddie Murphy se v tomto zábavném filmu stává zhýčkaným a sebevědomým africkým princem, který přichází do Ameriky, aby si tu na

In [363]:
vanocni_final.filter(~pl.col('id').is_null())

url,format,id_puvodni,nazev,zanry,zeme,rok,delka,rezie,scenar,hraji,anotace,hodnoceni,pocet_hodnoceni,id
str,str,str,str,list[str],str,i64,i64,list[str],list[str],list[str],str,i64,i64,i64
"""https://www.csfd.cz/film/4570-pelisky/prehled/""","""film""","""4570-pelisky""","""Pelíšky""","[""komedie"", ""drama""]","""Česko""",1999,116,"[""Jan Hřebejk""]","[""Petr Jarchovský""]","[""Miroslav Donutil"", ""Jiří Kodet"", … ""více""]","""Příběhy jedné historické generace - stárnoucích rodičů, dospívající mládeže a malých dětí. Děj je zasazen do konce šedesátých let - podzim 67 až léto …",91,117084,14
"""https://www.csfd.cz/film/821-s-certy-nejsou-zerty/prehled/""","""film""","""821-s-certy-nejsou-zerty""","""S čerty nejsou žerty""","[""pohádka"", ""komedie""]","""Československo""",1984,91,"[""Hynek Bočan""]","[""Jiří Just"", ""Hynek Bočan""]","[""Luděk Munzar"", ""Vladimír Dlouhý"", … ""více""]","""Chamtivá macecha připraví svého nevlastního syna Petra o všechno, včetně dědictví po jeho otci. Petr se mezitím seznámí s dcerami místního knížete, kd…",89,80866,21
"""https://www.csfd.cz/film/14999-shrek/prehled/""","""film""","""14999-shrek""","""Shrek""","[""animovaný"", ""dobrodružný"", … ""komedie""]","""USA""",2001,90,"[""Vicky Jenson"", ""Andrew Adamson""]","[""Ted Elliott"", ""Terry Rossio"", … ""Roger S.H. Schulman""]","[""Mike Myers"", ""Eddie Murphy"", … ""více""]","""Shrek, zelený obr s tykadly mimozemšťanů, žije sice docela sám, ale šťastně ve svém domku na samotě u lesa až do chvíle, kdy je doslova přepaden nekon…",88,101749,22
"""https://www.csfd.cz/film/6663-na-samote-u-lesa/prehled/""","""film""","""6663-na-samote-u-lesa""","""Na samotě u lesa""","[""komedie"", ""rodinný"", ""drama""]","""Československo""",1976,93,"[""Jiří Menzel""]","[""Zdeněk Svěrák"", ""Ladislav Smoljak""]","[""Josef Kemr"", ""Zdeněk Svěrák"", … ""více""]","""Hrdiny jsou manželé Oldřich a Věra Lavičkovi, kteří se svými dětmi Petříkem a Zuzankou odjíždějí do Loukova na chalupu svého známého, inženýra Radima …",88,66837,11
"""https://www.csfd.cz/film/8186-bohous/prehled/""","""tv""","""8186-bohous""","""Bohouš""","[""krátkometrážní"", ""komedie""]","""Československo""",1968,25,"[""Petr Schulhoff""]","[""František Vlček st.""]","[""Jiří Sovák"", ""Vladimír Menšík"", … ""pes Baldur""]","""Správce horského hotýlku Alois Randa (Jiří Sovák) má starostí nad hlavu: očekává početný zájezd turistů, nestíhá vařit a jeho jediná zaměstnankyně, po…",88,7462,2
"""https://www.csfd.cz/film/9793-nekdo-to-rad-horke/prehled/""","""film""","""9793-nekdo-to-rad-horke""","""Někdo to rád horké""","[""komedie"", ""krimi"", … ""hudební""]","""USA""",1959,120,"[""Billy Wilder""]","[""Billy Wilder"", ""I.A.L. Diamond""]","[""Marilyn Monroe"", ""Tony Curtis"", … ""více""]","""Slavná kombinace hudební komedie, parodie na gangsterské filmy a bláznivé crazy komedie, se odehrává v roce 1929 a jejími hrdiny jsou Joe a Jerry, dva…",88,37868,13
"""https://www.csfd.cz/film/9926-prazdniny-v-rime/prehled/""","""film""","""9926-prazdniny-v-rime""","""Prázdniny v Římě""","[""komedie"", ""romantický""]","""USA""",1953,118,"[""William Wyler""]","[""Dalton Trumbo"", ""Ian McLellan Hunter"", ""John Dighton""]","[""Gregory Peck"", ""Audrey Hepburn"", … ""více""]","""Romantická komedie s A. Hepburn a G. Peckem v hlavních rolích. Prázdniny v Římě byly nominovány na 10 Oscarů a Audrey Hepburn získala Oscara za svou r…",88,26697,18
"""https://www.csfd.cz/film/10089-byl-jednou-jeden-kral/prehled/""","""film""","""10089-byl-jednou-jeden-kral""","""Byl jednou jeden král...""","[""pohádka"", ""komedie""]","""Československo""",1954,107,"[""Bořivoj Zeman""]","[""Jan Werich"", ""Jiří Brdečka"", ""Bořivoj Zeman""]","[""Jan Werich"", ""Vlasta Burian"", … ""více""]","""Byl jednou jeden král a ten si říkal Já První. Chtěl odevzdat vládu té ze tří dcer, která ho má nejraději. Drahomíra ho má ráda jako zlato, Zpěvanka j…",87,56191,3
"""https://www.csfd.cz/film/1628-sam-doma/prehled/""","""film""","""1628-sam-doma""",

In [364]:
vanocni_final.sort(by='pocet_hodnoceni')

url,format,id_puvodni,nazev,zanry,zeme,rok,delka,rezie,scenar,hraji,anotace,hodnoceni,pocet_hodnoceni,id
str,str,str,str,list[str],str,i64,i64,list[str],list[str],list[str],str,i64,i64,i64
"""https://www.csfd.cz/film/5335-silvestr-svobodneho-pana/prehled/""","""tv""","""5335-silvestr-svobodneho-pana""","""Silvestr svobodného pána""","[""komedie""]","""Československo""",1979,31,"[""Jiří Krejčík""]","[""Jiří Krejčík"", ""Gustav Oplustil""]","[""Josef Abrhám"", ""Libuše Šafránková"", … ""Jana Vaňková""]","""Petr, mladý inženýr, má rozličné pletky s dívkami, od nedospělé Milušky v domě, kde žije, až po další, i vdané paničky. Chystá se zrovna na Silvestra,…",61,887,null
"""https://www.csfd.cz/film/22216-dunston-sam-v-hotelu/prehled/""","""film""","""22216-dunston-sam-v-hotelu""","""Dunston: Sám v hotelu""","[""komedie"", ""dobrodružný"", ""rodinný""]","""USA""",1996,85,"[""Ken Kwapis""]","[""John Hopkins"", ""Bruce Graham""]","[""Jason Alexander"", ""Faye Dunaway"", … ""více""]","""Nepřejte si vědět, co se stane, když se v pětihvězdičkovém hotelu objeví zloděj šperků se svým orangutanem. Ve chvíli, kdy se navíc čilá opička pánovi…",51,1132,null
"""https://www.csfd.cz/film/236348-o-kominickem-ucni-a-dceri-cukrare/prehled/""","""film""","""236348-o-kominickem-ucni-a-dceri-cukrare""","""O kominickém učni a dceři cukráře""","[""rodinný"", ""romantický""]","""Česko""",2007,63,"[""Pavel Jandourek""]","[""Václav Holanec"", ""Alena Bartošíková""]","[""Tomáš Materna"", ""Adriana Neubauerová"", … ""více""]","""Sirotek Tomáš (T. Materna) musel nastoupit do učení ke kominickému mistru Dýmalovi (P. Zedníček) a podřídit se tak rozhodnutí svého strýce. Začátky ne…",72,1763,null
"""https://www.csfd.cz/film/108846-o-medvedu-ondrejovi/prehled/""","""film""","""108846-o-medvedu-ondrejovi""","""O medvědu Ondřejovi""","[""pohádka"", ""komedie""]","""Československo""",1959,50,"[""Jaroslav Mach""]",null,"[""Jaroslav Marvan"", ""Aglaia Morávková"", … ""více""]","""Princezna Blanka se tajně schází se svým milým, myslivcem Ondřejem. Jejich lásce přeje jen komorná Anežka, královský otec se nesmí nic dozvědět. Blank…",59,1851,null
"""https://www.csfd.cz/film/111222-kotva-u-privozu/prehled/""","""tv""","""111222-kotva-u-privozu""","""Kotva u přívozu""","[""komedie"", ""drama""]","""Československo""",1980,90,"[""Marie Poledňáková""]","[""Marie Poledňáková""]","[""Jana Šulcová"", ""Petr Haničinec"", … ""více""]","""Magda je televizní hlasatelka. Pro miliony diváků příjemná, sympatická tvář na obrazovce. Klidná, úsměvná, vyrovnaná. Příchod pohody do rodinného veče…",60,2020,null
"""https://www.csfd.cz/film/3118-kral-kralu/prehled/""","""film""","""3118-kral-kralu""","""Král Králů""","[""komedie""]","""Československo""",1963,100,"[""Martin Frič""]","[""Jiří Mucha""]","[""Jiří Sovák"", ""Miloš Kopecký"", … ""více""]","""Uprostřed pouště nalezne osamělý jezdec na oslu starého muže, který leží v bezvědomí ve stínu luxusního automobilu. Jezdec naloží starce na osla a odv…",65,2645,null
"""https://www.csfd.cz/film/8496-falesna-kocicka/prehled/""","""film""","""8496-falesna-kocicka""","""Falešná kočička""","[""komedie""]","""Československo""",1937,107,"[""Vladimír Slavínský""]","[""Karel Melíšek"", ""Vladimír Slavínský"", ""Josef Skružný""]","[""Věra Ferbasová"", ""Karel Jičínský"", … ""více""]","""MUDr. Přeloučovi předpoví kartářka, že se ožení s třetím chudým děvčetem, které potká. Míle, dceři továrníka Janoty, se doktor líbí a s pomocí jeho ho…",69,2971,null
"""https://www.csfd.cz/film/197358-dalskabaty-hrisna-ves-aneb-zapomenuty-cert/prehled/""","""tv""","""197358-dalskabaty-hrisna-ves-aneb-zapomenuty-cert""","""Dalskabáty, hříšná ves aneb Zapomenutý čert""","[""pohádka"", ""komedie""]","""Československo""",1976,116,"[""Jaroslav Novotný""]","[""Jan Drda"", ""Jaroslav Novotný""]","[""Jiřina Bohdalová"", ""Jaroslav Moučka"", … ""více""]","""Hubatá, rázná Plajznerka (J. Bohdalová), jíž hraje v těle snad každá žilka touhou po hutném krajíci štěstí, krotí a cepuje zapom

In [365]:
vanocni_final.filter(pl.col('pocet_hodnoceni') < 26000)

url,format,id_puvodni,nazev,zanry,zeme,rok,delka,rezie,scenar,hraji,anotace,hodnoceni,pocet_hodnoceni,id
str,str,str,str,list[str],str,i64,i64,list[str],list[str],list[str],str,i64,i64,i64
"""https://www.csfd.cz/film/8186-bohous/prehled/""","""tv""","""8186-bohous""","""Bohouš""","[""krátkometrážní"", ""komedie""]","""Československo""",1968,25,"[""Petr Schulhoff""]","[""František Vlček st.""]","[""Jiří Sovák"", ""Vladimír Menšík"", … ""pes Baldur""]","""Správce horského hotýlku Alois Randa (Jiří Sovák) má starostí nad hlavu: očekává početný zájezd turistů, nestíhá vařit a jeho jediná zaměstnankyně, po…",88,7462,2
"""https://www.csfd.cz/film/5475-duchacek-to-zaridi/prehled/""","""film""","""5475-duchacek-to-zaridi""","""Ducháček to zařídí""","[""komedie""]","""Československo""",1938,86,"[""Karel Lamač""]","[""Václav Wasserman"", ""Rudolf Österreicher"", … ""více""]","[""Vlasta Burian"", ""Ladislav Hemmer"", … ""více""]","""Pan Ducháček je sice poněkud svérázný patron, nicméně jako koncipient advokátní kanceláře doktora Faulknera je naprosto nenahraditelný. Zejména v okam…",86,21640,null
"""https://www.csfd.cz/film/3083-anton-spelec-ostrostrelec/prehled/""","""film""","""3083-anton-spelec-ostrostrelec""","""Anton Špelec, ostrostřelec""","[""komedie""]","""Československo""",1932,80,"[""Martin Frič""]","[""Josef Neuberg"", ""Emil Artur Longen""]","[""Vlasta Burian"", ""Růžena Šlemrová"", … ""více""]","""Nejzdařilejší ze série burianovských komedií, natočených v třicátých letech, pojednává o tom, co způsobila urážka císařského majestátu, a o snaze vyhn…",82,10407,null
"""https://www.csfd.cz/film/197358-dalskabaty-hrisna-ves-aneb-zapomenuty-cert/prehled/""","""tv""","""197358-dalskabaty-hrisna-ves-aneb-zapomenuty-cert""","""Dalskabáty, hříšná ves aneb Zapomenutý čert""","[""pohádka"", ""komedie""]","""Československo""",1976,116,"[""Jaroslav Novotný""]","[""Jan Drda"", ""Jaroslav Novotný""]","[""Jiřina Bohdalová"", ""Jaroslav Moučka"", … ""více""]","""Hubatá, rázná Plajznerka (J. Bohdalová), jíž hraje v těle snad každá žilka touhou po hutném krajíci štěstí, krotí a cepuje zapomenutého čerta, aby z n…",80,3101,4
"""https://www.csfd.cz/film/3115-katakomby/prehled/""","""film""","""3115-katakomby""","""Katakomby""","[""komedie""]","""Protektorát Čechy a Morava""",1940,80,"[""Martin Frič""]","[""Václav Wasserman"", ""Gustav Davis""]","[""Vlasta Burian"", ""Jaroslav Marvan"", … ""více""]","""Starý úředník, oficiál Borman, úřaduje ve sklepních kancelářích Pozemkového úřadu společně s dalšími oficiály a aktuáry. Mezi jeho kolegy je i mladý a…",79,5661,null
"""https://www.csfd.cz/film/3166-to-neznate-hadimrsku/prehled/""","""film""","""3166-to-neznate-hadimrsku""","""To neznáte Hadimršku""","[""komedie""]","""Československo""",1931,91,"[""Karel Lamač"", ""Martin Frič""]","[""Václav Wasserman"", ""Franz Arnold"", … ""více""]","[""Vlasta Burian"", ""Meda Valentová"", … ""více""]","""Revident Hadimrška je vyslán do Prahy. Má reorganizovat Zlatníkovu gramofonovou firmu, aby mohla bance zaplatit dluhy. Mladý majitel se zruinoval vydr…",79,9278,null
"""https://www.csfd.cz/film/4238-panna-a-netvor/prehled/""","""film""","""4238-panna-a-netvor""","""Panna a netvor""","[""pohádka"", ""horor"", … ""romantický""]","""Československo""",1978,88,"[""Juraj Herz""]","[""Juraj Herz"", ""Ota Hofman"", ""František Hrubín""]","[""Zdena Studenková"", ""Vlastimil Harapes"", … ""více""]","""Slavnou strašidelnou pohádku, původně francouzskou, o princi začarovaném do děsivé zvířecí podoby, jehož vysvobodí až čistá láska, využil režisér Jura…",78,10311,null
"""https://www.csfd.cz/film/9411-jak-se-budi-princezny/prehled/""","""film""","""9411-jak-se-budi-princezny""","""Jak se budí princezny""","[""pohádka""]","""Československo""",1977,83,"[""Václav Vorlíček""]","[""Bohumila Zelenková""]","[""Jiří Sovák"", ""Milena Dvorská"", … ""více""]","""Ještě před narozením královské dcery jedou král s královnou do zpustlé tvrze - v ní žije královnina sestra, už spoustu let odlou

In [366]:
musi_byt = [
    "Bohouš",
    "Dalskabáty, hříšná ves aneb Zapomenutý čert",
    "Chobotnice z II. patra",
    "Veselé Vánoce přejí chobotnice",
    "Nesmrtelná teta",
    "Láska rohatá"
]

nebude = ['Shrek 2',"Chobotnice z II. patra","Sám doma 2: Ztracen v New Yorku"]

In [367]:
vanocni_final = vanocni_final.filter(
    (pl.col('pocet_hodnoceni') >= 26000) | (pl.col('nazev').is_in(musi_byt))
).filter(
    ~pl.col('nazev').is_in(nebude)
).sort(
    by='pocet_hodnoceni',descending=True
).with_columns(
    pl.when(pl.col("zeme").str.contains("Česk")).then(pl.lit('domaci')).otherwise(pl.lit('zahranicni')).alias("domaci")
).with_columns(
    pl.when(pl.col('nazev').is_in(animace)).then(pl.lit('animovany')).otherwise(pl.lit('hrany')).alias('animace')
).with_columns(
    pl.when(pl.col('nazev').is_in(travesti)).then(pl.lit('crossdressing')).otherwise(pl.lit('cisdressing')).alias('travesti')
).with_columns(
    pl.when(pl.col('nazev').is_in(nadprirozene)).then(pl.lit('nadprirozene')).otherwise(pl.lit('realisticke')).alias('nadprirozene')
).with_columns(
    pl.when(pl.col('nazev').is_in(jesex)).then(pl.lit('je')).otherwise(pl.lit('neni')).alias('jesex')
)

vanocni_final

url,format,id_puvodni,nazev,zanry,zeme,rok,delka,rezie,scenar,hraji,anotace,hodnoceni,pocet_hodnoceni,id,domaci,animace,travesti,nadprirozene,jesex
str,str,str,str,list[str],str,i64,i64,list[str],list[str],list[str],str,i64,i64,i64,str,str,str,str,str
"""https://www.csfd.cz/film/4570-pelisky/prehled/""","""film""","""4570-pelisky""","""Pelíšky""","[""komedie"", ""drama""]","""Česko""",1999,116,"[""Jan Hřebejk""]","[""Petr Jarchovský""]","[""Miroslav Donutil"", ""Jiří Kodet"", … ""více""]","""Příběhy jedné historické generace - stárnoucích rodičů, dospívající mládeže a malých dětí. Děj je zasazen do konce šedesátých let - podzim 67 až léto …",91,117084,14,"""domaci""","""hrany""","""cisdressing""","""realisticke""","""je"""
"""https://www.csfd.cz/film/14999-shrek/prehled/""","""film""","""14999-shrek""","""Shrek""","[""animovaný"", ""dobrodružný"", … ""komedie""]","""USA""",2001,90,"[""Vicky Jenson"", ""Andrew Adamson""]","[""Ted Elliott"", ""Terry Rossio"", … ""Roger S.H. Schulman""]","[""Mike Myers"", ""Eddie Murphy"", … ""více""]","""Shrek, zelený obr s tykadly mimozemšťanů, žije sice docela sám, ale šťastně ve svém domku na samotě u lesa až do chvíle, kdy je doslova přepaden nekon…",88,101749,22,"""zahranicni""","""animovany""","""cisdressing""","""nadprirozene""","""neni"""
"""https://www.csfd.cz/film/821-s-certy-nejsou-zerty/prehled/""","""film""","""821-s-certy-nejsou-zerty""","""S čerty nejsou žerty""","[""pohádka"", ""komedie""]","""Československo""",1984,91,"[""Hynek Bočan""]","[""Jiří Just"", ""Hynek Bočan""]","[""Luděk Munzar"", ""Vladimír Dlouhý"", … ""více""]","""Chamtivá macecha připraví svého nevlastního syna Petra o všechno, včetně dědictví po jeho otci. Petr se mezitím seznámí s dcerami místního knížete, kd…",89,80866,21,"""domaci""","""hrany""","""cisdressing""","""nadprirozene""","""neni"""
"""https://www.csfd.cz/film/1628-sam-doma/prehled/""","""film""","""1628-sam-doma""","""Sám doma""","[""rodinný"", ""komedie""]","""USA""",1990,103,"[""Chris Columbus""]","[""John Hughes""]","[""Macaulay Culkin"", ""Joe Pesci"", … ""více""]","""""Mám pocit, že jsme doma nechali něco důležitého,"" sdělí paní McCallisterová manželovi na palubě letadla směřujícího přes Atlantik do Paříže. To ještě…",86,80771,24,"""zahranicni""","""hrany""","""cisdressing""","""realisticke""","""neni"""
"""https://www.csfd.cz/film/43677-laska-nebeska/prehled/""","""film""","""43677-laska-nebeska""","""Láska nebeská""","[""komedie"", ""drama"", ""romantický""]","""Velká Británie / USA / Francie""",2003,135,"[""Richard Curtis""]","[""Richard Curtis""]","[""Bill Nighy"", ""Gregor Fisher"", … ""více""]","""Láska nebeská, romantická komedie ve stylu Čtyři svatby a jeden pohřeb, Notting Hill a Deník Bridget Jonesové, nás zavede do současného předvánočního …",85,76558,8,"""zahranicni""","""hrany""","""cisdressing""","""realisticke""","""je"""
"""https://www.csfd.cz/film/9430-tri-orisky-pro-popelku/prehled/""","""film""","""9430-tri-orisky-pro-popelku""","""Tři oříšky pro Popelku""","[""pohádka""]","""Československo / Východní Německo""",1973,75,"[""Václav Vorlíček""]","[""František Pavlíček"", ""Václav Vorlíček""]","[""Libuše Šafránková"", ""Pavel Trávníček"", … ""více""]","""Popelka je milé hodné děvče bez matky a posléze i bez otce, který ji nechá napospas své druhé ženě a její dceři. Obě nemají Popelku rády a mimo jiné s…",86,73324,26,"""domaci""","""hrany""","""crossdressing""","""nadprirozene""","""neni"""
"""https://www.csfd.cz/film/6663-na-samote-u-lesa/prehled/""","""film""","""6663-na-samote-u-lesa""","""Na samotě u lesa""","[""komedie"", ""rodinný"", ""drama""]","""Československo""",1976,93,"[""Jiří Menzel""]","[""Zdeněk Svěrák"", ""Ladislav Smoljak""]","[""Josef Kemr"", ""Zdeněk Svěrák"", … ""více""]","""Hrdiny jsou manželé Oldřich a Věra Lavičkovi, kteří se svými dětmi Petříkem a Zuzankou odjíždějí do Loukova na chalupu svého známého, inženýra Radima …",88,66837,11,"""domaci""","""hrany""","""cisdressing""","""realisticke""","""neni"""
"

In [368]:
vanocni_final.sort(by="nazev").write_json("data/vanocni_filmy_kompletni_metadata_vyber.json")

In [369]:
vanocni_final.sort(by="nazev")

url,format,id_puvodni,nazev,zanry,zeme,rok,delka,rezie,scenar,hraji,anotace,hodnoceni,pocet_hodnoceni,id,domaci,animace,travesti,nadprirozene,jesex
str,str,str,str,list[str],str,i64,i64,list[str],list[str],list[str],str,i64,i64,i64,str,str,str,str,str
"""https://www.csfd.cz/film/5981-adela-jeste-nevecerela/prehled/""","""film""","""5981-adela-jeste-nevecerela""","""Adéla ještě nevečeřela""","[""komedie"", ""krimi"", ""sci-fi""]","""Československo""",1978,105,"[""Oldřich Lipský""]","[""Jiří Brdečka"", ""Oldřich Lipský""]","[""Michal Dočolomanský"", ""Rudolf Hrušínský"", … ""více""]","""Neporazitelný soukromý detektiv Nick Carter přijíždí do Prahy řešit případ, s nímž si zdejší policie neví rady. Je vyzbrojen geniálním mozkem a nejnov…",85,40792,0,"""domaci""","""animovany""","""cisdressing""","""nadprirozene""","""neni"""
"""https://www.csfd.cz/film/215414-andel-pane/prehled/""","""film""","""215414-andel-pane""","""Anděl Páně""","[""pohádka""]","""Česko""",2005,91,"[""Jiří Strach""]","[""Lucie Konášová""]","[""Ivan Trojan"", ""Jiří Dvořák"", … ""více""]","""Je časné ráno, v předvečer Štědrého dne, a celé nebe se chystá na velkolepé oslavy Ježíškových narozenin. Panna Marie (Klára Issová) se svatou Veronik…",77,40797,1,"""domaci""","""hrany""","""cisdressing""","""nadprirozene""","""neni"""
"""https://www.csfd.cz/film/8186-bohous/prehled/""","""tv""","""8186-bohous""","""Bohouš""","[""krátkometrážní"", ""komedie""]","""Československo""",1968,25,"[""Petr Schulhoff""]","[""František Vlček st.""]","[""Jiří Sovák"", ""Vladimír Menšík"", … ""pes Baldur""]","""Správce horského hotýlku Alois Randa (Jiří Sovák) má starostí nad hlavu: očekává početný zájezd turistů, nestíhá vařit a jeho jediná zaměstnankyně, po…",88,7462,2,"""domaci""","""hrany""","""cisdressing""","""realisticke""","""neni"""
"""https://www.csfd.cz/film/10089-byl-jednou-jeden-kral/prehled/""","""film""","""10089-byl-jednou-jeden-kral""","""Byl jednou jeden král...""","[""pohádka"", ""komedie""]","""Československo""",1954,107,"[""Bořivoj Zeman""]","[""Jan Werich"", ""Jiří Brdečka"", ""Bořivoj Zeman""]","[""Jan Werich"", ""Vlasta Burian"", … ""více""]","""Byl jednou jeden král a ten si říkal Já První. Chtěl odevzdat vládu té ze tří dcer, která ho má nejraději. Drahomíra ho má ráda jako zlato, Zpěvanka j…",87,56191,3,"""domaci""","""hrany""","""cisdressing""","""nadprirozene""","""neni"""
"""https://www.csfd.cz/film/3094-cisaruv-pekar-pekaruv-cisar/prehled/""","""film""","""3094-cisaruv-pekar-pekaruv-cisar""","""Císařův pekař""","[""komedie"", ""historický""]","""Československo""",1951,80,"[""Martin Frič""]","[""Jan Werich"", ""Martin Frič"", … ""více""]","[""Jan Werich"", ""Marie Vášová"", … ""více""]","""Historická veselohra o císaři Rudolfovi II., pekaři Matějovi a legendárním Golemovi. Tento příběh neodpovídá přesně historické pravdě, ale je zato nad…",86,55421,25,"""domaci""","""hrany""","""cisdressing""","""nadprirozene""","""neni"""
"""https://www.csfd.cz/film/197358-dalskabaty-hrisna-ves-aneb-zapomenuty-cert/prehled/""","""tv""","""197358-dalskabaty-hrisna-ves-aneb-zapomenuty-cert""","""Dalskabáty, hříšná ves aneb Zapomenutý čert""","[""pohádka"", ""komedie""]","""Československo""",1976,116,"[""Jaroslav Novotný""]","[""Jan Drda"", ""Jaroslav Novotný""]","[""Jiřina Bohdalová"", ""Jaroslav Moučka"", … ""více""]","""Hubatá, rázná Plajznerka (J. Bohdalová), jíž hraje v těle snad každá žilka touhou po hutném krajíci štěstí, krotí a cepuje zapomenutého čerta, aby z n…",80,3101,4,"""domaci""","""hrany""","""cisdressing""","""nadprirozene""","""neni"""
"""https://www.csfd.cz/film/3095-darbujan-a-pandrhola/prehled/""","""film""","""3095-darbujan-a-pandrhola""","""Dařbuján a Pandrhola""","[""pohádka"", ""komedie""]","""Československo""",1959,80,"[""Martin Frič""]","[""Jan Drda"", ""Martin Frič""]","[""Jiří Sovák"", ""Rudolf Hrušínský"", … ""více""]","""Hrdinou je chudý havíř Kuba Dařbuján, který se stará o svou Markýtku a jedenáct dětí. Není divu, že se jen málokdy nají dos

In [370]:
len(vanocni_final)

30

In [371]:
vanocni_final.select(pl.col("nazev")).sort(by="nazev")

nazev
str
"""Adéla ještě nevečeřela"""
"""Anděl Páně"""
"""Bohouš"""
"""Byl jednou jeden král..."""
"""Císařův pekař"""
"""Dalskabáty, hříšná ves aneb Zapomenutý čert"""
"""Dařbuján a Pandrhola"""
"""Dobrý voják Švejk"""
"""Grinch"""


In [372]:
vanocni_final.sample(2)

url,format,id_puvodni,nazev,zanry,zeme,rok,delka,rezie,scenar,hraji,anotace,hodnoceni,pocet_hodnoceni,id,domaci,animace,travesti,nadprirozene,jesex
str,str,str,str,list[str],str,i64,i64,list[str],list[str],list[str],str,i64,i64,i64,str,str,str,str,str
"""https://www.csfd.cz/film/4570-pelisky/prehled/""","""film""","""4570-pelisky""","""Pelíšky""","[""komedie"", ""drama""]","""Česko""",1999,116,"[""Jan Hřebejk""]","[""Petr Jarchovský""]","[""Miroslav Donutil"", ""Jiří Kodet"", … ""více""]","""Příběhy jedné historické generace - stárnoucích rodičů, dospívající mládeže a malých dětí. Děj je zasazen do konce šedesátých let - podzim 67 až léto …",91,117084,14,"""domaci""","""hrany""","""cisdressing""","""realisticke""","""je"""
"""https://www.csfd.cz/film/6486-pretty-woman/prehled/""","""film""","""6486-pretty-woman""","""Pretty Woman""","[""komedie"", ""romantický""]","""USA""",1990,119,"[""Garry Marshall""]","[""J.F. Lawton""]","[""Richard Gere"", ""Julia Roberts"", … ""více""]","""Hollywood. Vivian šlape chodník a sní o princi na bílém koni. Jednoho večera se náhodnou seznámí s krásným, osamělým a trochu tajemným milionářem Edwa…",79,54349,15,"""zahranicni""","""hrany""","""cisdressing""","""realisticke""","""je"""


In [373]:
df_pro_model = vanocni_final.with_columns(pl.col("zeme").str.split(" / ")).with_columns(
    (pl.col('rok').cast(str).str.slice(0,3) + pl.lit('0')).alias('dekada')
)

neseznamove_sloupce = ['format','dekada','domaci','animace','travesti','nadprirozene','jesex']
seznamove_sloupce = ['zeme','zanry','rezie','scenar','hraji']

for s in neseznamove_sloupce:
    seznam_moznosti = df_pro_model.select(pl.col(s)).unique().to_series().to_list()
    for m in seznam_moznosti:
        nazev = f'{s}_{m}'.replace(' ','_')
        df_pro_model = df_pro_model.with_columns(pl.lit(None).alias(nazev))
        df_pro_model = df_pro_model.with_columns(pl.when(pl.col(s) == m).then(pl.lit(1)).otherwise(pl.col(nazev)).alias(nazev))

for s in seznamove_sloupce:
    seznam_moznosti = df_pro_model.explode(s).select(pl.col(s)).unique().to_series().to_list()
    for m in seznam_moznosti:
        nazev = f'{s}_{m}'.replace(' ','_')
        df_pro_model = df_pro_model.with_columns(pl.lit(None).alias(nazev))
        df_pro_model = df_pro_model.with_columns(pl.when(pl.col(s).list.contains(m)).then(pl.lit(1)).otherwise(pl.col(nazev)).alias(nazev))

df_pro_model = df_pro_model.fill_null(
    0
).drop(
    ["url","format",'zanry','zeme','rezie','scenar','hraji','anotace','dekada','id_puvodni','domaci','animace','travesti','nadprirozene','jesex']
).drop(
    cs.contains("více")
)

In [374]:
df_pro_model.columns

['nazev',
 'rok',
 'delka',
 'hodnoceni',
 'pocet_hodnoceni',
 'id',
 'format_tv',
 'format_film',
 'dekada_1980',
 'dekada_1970',
 'dekada_2000',
 'dekada_1950',
 'dekada_1990',
 'dekada_1960',
 'domaci_domaci',
 'domaci_zahranicni',
 'animace_animovany',
 'animace_hrany',
 'travesti_cisdressing',
 'travesti_crossdressing',
 'nadprirozene_realisticke',
 'nadprirozene_nadprirozene',
 'jesex_neni',
 'jesex_je',
 'zeme_Velká_Británie',
 'zeme_Československo',
 'zeme_Německo',
 'zeme_Sovětský_svaz',
 'zeme_USA',
 'zeme_Západní_Německo',
 'zeme_Francie',
 'zeme_Východní_Německo',
 'zeme_Česko',
 'zanry_historický',
 'zanry_thriller',
 'zanry_válečný',
 'zanry_rodinný',
 'zanry_sci-fi',
 'zanry_pohádka',
 'zanry_drama',
 'zanry_hudební',
 'zanry_krimi',
 'zanry_dobrodružný',
 'zanry_akční',
 'zanry_komedie',
 'zanry_fantasy',
 'zanry_romantický',
 'zanry_krátkometrážní',
 'zanry_animovaný',
 'rezie_Jan_Hřebejk',
 'rezie_Oldřich_Lipský',
 'rezie_Zdeněk_Zelenka',
 'rezie_Billy_Wilder',
 'rezi

In [375]:
df_pro_model.select(pl.col("nazev"))

nazev
str
"""Pelíšky"""
"""Shrek"""
"""S čerty nejsou žerty"""
"""Sám doma"""
"""Láska nebeská"""
"""Tři oříšky pro Popelku"""
"""Na samotě u lesa"""
"""Byl jednou jeden král..."""
"""Císařův pekař"""


In [376]:
os.makedirs("data/vanocni_filmy_pro_model",exist_ok=True)

In [377]:
df_pro_model.write_csv("data/vanocni_filmy_pro_model/vanocni_filmy_pro_model_komplet.csv")

In [378]:
for i in range(2,9):
    df_pro_model_pracovni = df_pro_model.clone()
    for sloupec in df_pro_model.columns:
        if ('_' in sloupec) and ('_více' not in sloupec):
            soucet = df_pro_model.select(pl.col(sloupec).sum()).item()
            if soucet < i:
                df_pro_model_pracovni = df_pro_model_pracovni.drop(sloupec)
    df_pro_model_pracovni.write_csv(f"data/vanocni_filmy_pro_model/vanocni_filmy_pro_model_n{i}.csv")
    print(f'Uloženo: n={i}, počet sloupců={len(df_pro_model_pracovni.columns)}')

Uloženo: n=2, počet sloupců=172
Uloženo: n=3, počet sloupců=79
Uloženo: n=4, počet sloupců=50
Uloženo: n=5, počet sloupců=32
Uloženo: n=6, počet sloupců=23
Uloženo: n=7, počet sloupců=20
Uloženo: n=8, počet sloupců=20


In [379]:
df_pro_model_pracovni

nazev,rok,delka,hodnoceni,pocet_hodnoceni,id,format_film,dekada_1950,domaci_domaci,domaci_zahranicni,animace_hrany,travesti_cisdressing,nadprirozene_realisticke,nadprirozene_nadprirozene,jesex_neni,zeme_Československo,zeme_USA,zanry_rodinný,zanry_pohádka,zanry_komedie
str,i64,i64,i64,i64,i64,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32
"""Pelíšky""",1999,116,91,117084,14,1,0,1,0,1,1,1,0,0,0,0,0,0,1
"""Shrek""",2001,90,88,101749,22,1,0,0,1,0,1,0,1,1,0,1,1,0,1
"""S čerty nejsou žerty""",1984,91,89,80866,21,1,0,1,0,1,1,0,1,1,1,0,0,1,1
"""Sám doma""",1990,103,86,80771,24,1,0,0,1,1,1,1,0,1,0,1,1,0,1
"""Láska nebeská""",2003,135,85,76558,8,1,0,0,1,1,1,1,0,0,0,1,0,0,1
"""Tři oříšky pro Popelku""",1973,75,86,73324,26,1,0,1,0,1,0,0,1,1,1,0,0,1,0
"""Na samotě u lesa""",1976,93,88,66837,11,1,0,1,0,1,1,1,0,1,1,0,1,0,1
"""Byl jednou jeden král...""",1954,107,87,56191,3,1,1,1,0,1,1,0,1,1,1,0,0,1,1
"""Císařův pekař""",1951,80,86,55421,25,1,1,1,0,1,1,0,1,1,1,0,0,0,1
